In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# =========================
# تنظیمات
# =========================
INPUT_FILE = "merged_output2.csv"
OUTPUT_FILE = "sarima_declared_output.csv"

DATE_COL = "DATE_MILADI"
TARGET_COL = "POWER"
EBRAZ_COL = "ebraz"

# =========================
# خواندن داده
# =========================
df = pd.read_csv(INPUT_FILE)
df[DATE_COL] = pd.to_datetime(df[DATE_COL])

df = df.sort_values(DATE_COL)

# =========================
# ساخت سری زمانی ساعتی
# =========================
ts = df.set_index(DATE_COL)[TARGET_COL]
ts = ts.asfreq("H")

# =========================
# Train / Test
# =========================
train_ts = ts["2021-01-01":"2023-12-31"]
test_df  = df[df[DATE_COL] >= "2024-01-01"]

# =========================
# آموزش SARIMA (بدون differencing)
# =========================
model = SARIMAX(
    train_ts,
    order=(1, 0, 1),
    seasonal_order=(1, 0, 1, 24),
    trend="c",   # حفظ سطح (خیلی مهم)
    enforce_stationarity=False,
    enforce_invertibility=False
)

model_fit = model.fit(disp=False)

# =========================
# پیش‌بینی
# =========================
forecast = model_fit.predict(
    start=test_df[DATE_COL].min(),
    end=test_df[DATE_COL].max()
)

forecast = forecast.reindex(test_df[DATE_COL])
forecast = np.maximum(forecast.values, 0)

# =========================
# ساخت خروجی
# =========================
output_df = test_df.copy()

output_df["year"] = output_df[DATE_COL].dt.year
output_df["month"] = output_df[DATE_COL].dt.month
output_df["dayofweek"] = output_df[DATE_COL].dt.dayofweek

output_df["DECLARED"] = forecast

# =========================
# قانون بازار برق
# =========================
output_df.loc[output_df[EBRAZ_COL] == 0, "DECLARED"] = 0

# =========================
# ذخیره
# =========================
final_columns = [
    "HOUR", "DATE_MILADI", "DATE_SHAMSI", "POWER", "CODE", "UNIT_NO",
    "DAMA", "ROTOOBAT", "12209_G13", "12210_G13",
    "ebraz", "importance_factor",
    "year", "month", "dayofweek", "DECLARED"
]

output_df[final_columns].to_csv(OUTPUT_FILE, index=False)

print("✅ خروجی صحیح SARIMA ساخته شد:", OUTPUT_FILE)


C:\Users\pishva_r\AppData\Local\Temp\ipykernel_8288\615581912.py:27: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ts = ts.asfreq("H")
d:\ProgramData\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


✅ خروجی صحیح SARIMA ساخته شد: sarima_declared_output.csv
